In [1]:
%load_ext autoreload
%autoreload 2

<h3> Загрузка библиотек

In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR

In [3]:
#from ml.models.unet3d import U_Net
#from ml.models.rog import ROG
from ml.models.unet_deepsup import Unet_MSS
#from ml.models.LiquidNet import LiquidNet, LiquidNetBlock, conv_block, bottle_neck_connection
#from ml.models.LiquidNet_configs.second_config import get_config
from ml.models.ResUnet import ResUNet

from ml.models.activates import swish
from ml.utils import get_total_params, load_pretrainned, test_model
from ml.tio_dataset import TioDataset
from ml.controller import Controller
from ml.losses import (ExponentialLogarithmicLoss, WeightedExpBCE, TverskyLoss,
                       IOU_Metric, MultyscaleLoss, SumLoss, LinearCombLoss)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [14]:
train_settings  = {
    "patch_shape" : (64, 64, 64),
    "patches_per_volume" : 128,
    "patches_queue_length" : 1440,
    "batch_size" : 4,
    "num_workers": 4,
    "sampler": "weighted" #"uniform",#
}

val_settings  = {
    "patch_shape" : (64, 64, 64),
    "patches_per_volume" : 32,
    "patches_queue_length" : 1440,
    "batch_size" : 8,
    "num_workers": 4,
    "sampler": "uniform",#"weighted" #"uniform",#
}

test_settings  = {
    "patch_shape" : (128, 128, 128),
    "overlap_shape" : (32, 32, 24),
    "batch_size" : 1,
    "num_workers": 4,
}

data_dir = "/home/msst/Documents/medtech/MainData"
dataset = TioDataset(data_dir,
                 train_settings=train_settings,
                 val_settings=val_settings,
                 test_settings=test_settings)

In [7]:
# test_settings  = {
#     "patch_shape" : (128, 128, 128),
#     "overlap_shape" : (32, 32, 24),
#     "batch_size" : 1,
#     "num_workers": 4,
# }

# data_dir = "/home/msst/Documents/medtech/MainData_test"
# dataset = TioDataset(data_dir,
#                      train_settings=None,
#                      val_settings=None,
#                      test_settings=test_settings)

In [8]:
#model = Unet_MSS(channel_coef=16, act_fn=swish())

In [9]:
#model = LiquidNet(*get_config(16), debug=False)
#print("total_params:", get_total_params(model))
#model.net_graph

In [4]:
model = ResUNet(in_channels=1, 
                channels_coef=24, with_attention=True)
print("total_params:", get_total_params(model))

total_params: 22827390


In [6]:
#test_model(model)

In [17]:
# funcs_and_сoef_list = []

# funcs_and_сoef_list.append([ExponentialLogarithmicLoss(gamma_tversky = 1, gamma_bce = 1, lamb=0.0,
#                                    freq = 0.001, tversky_alfa=0.75), 1])

# funcs_and_сoef_list.append([TverskyLoss(0.55), 1])


# funcs_and_сoef_list.append([SumLoss(alfa=0.5), 0.1])

# loss_fn = MultyscaleLoss(LinearCombLoss(funcs_and_сoef_list))

In [18]:
#loss_fn = MultyscaleLoss(TverskyLoss(0.55))

In [11]:
loss_fn = MultyscaleLoss(ExponentialLogarithmicLoss(gamma_tversky=1, gamma_bce=1, lamb=0.9,
                                                    freq = 0.001, tversky_alfa=0.55))


metric_fn = IOU_Metric()

controller_config = {
    "loss" : loss_fn,
    "metric" : metric_fn,
    'device' : device,
    "optimizer_fn" : lambda model: torch.optim.Adam(model.parameters(), lr=0.01),
    "sheduler_fn": lambda optimizer: StepLR(optimizer, step_size=5, gamma=0.5)
}
controller = Controller(controller_config)

cuda


In [15]:
controller.fit(model, dataset, 50)

Epoch 22/71


100%|█████████████████████████████████████████| 128/128 [00:50<00:00,  2.56it/s]


{'mean_loss': nan}


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  4.39it/s]


{'mean_loss': nan}


...

{'mean_loss': nan}


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.01it/s]


{'mean_loss': nan}


  0%|                                                     | 0/2 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model_name = "ResUnet"
controller.save("/home/msst/save_folder/saved_models/" + model_name)

In [ ]:
path_to_check= "/home/msst/save_folder/saved_models/" + model_name
controller.load(model, path_to_checkpoint=path_to_check)

In [ ]:
controller.val_epoch(dataset.test_dataloader)

In [14]:
#model_name = "ResUnet"
data_dir = "/home/msst/save_folder/seg_data/" + model_name
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
controller.predict(dataset.test_dataloader, data_dir)

100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.08s/it]


[{'sample': 'P62_CTA_0',
  'seg_sum/GT_sum': tensor(0.1191),
  'metric1': tensor([2.9943e-11])},
 {'sample': 'new_CTA_0',
  'seg_sum/GT_sum': tensor(0.0267),
  'metric1': tensor([3.3001e-11])}]